In [ ]:
!py -m pip install openai
!py -m pip install pinecone-client
!py -m pip install "pinecone-client[grpc]"
!py -m pip install langchain
!py -m pip install numpy
!py -m pip install langchain

In [1]:
import pandas as pd
df = pd.read_csv("law.csv")
df['Content'] = df['Content'].str.replace("\n", " ")
df

,Content
0,Chapter 1: Terminology Section 1. Terminology ...
1,Chapter 2: Application of Criminal Law Section...
2,Chapter 3: Punishments and Measures of Safety ...
3,Chapter 3: Punishments and Measures of Safety ...
4,Chapter 3: Punishments and Measures of Safety ...
...,...
70,Section 367. Refusal to Provide Information to...
71,Section 375. Obstruction to a Public Gutter or...
72,Section 381. Animal Cruelty Any person who mal...
73,Section 385. Obstruction of Public Way by Leav...


In [ ]:
import pandas as pd
import os
# path = os.path.abspath("/content/drive/MyDrive/work/criminal-datasets.csv/")
df = pd.read_csv("criminal-datasets.csv")
df['text'] = df['text'].astype(str) + df['notes'].astype(str)
df['text'] = df['text'].str.replace('nan', '')
df = df.drop(['article','notes'], axis = 1)
df = df.drop(df.index[0:9])
df = df.reset_index()
df = df.drop('index', axis = 1)
df

In [1]:
import pinecone
from tqdm.auto import tqdm
import openai
import numpy as np
openai.api_type = "azure"
openai.api_base = "https://fusion-chatbot-demo.openai.azure.com/"
openai.api_version = "2023-05-15"
openai.api_key = "f07d23e10600494f8235496bbf5054b9"
pinecone.init(api_key="d9b8d145-bab6-406a-ac8a-e3a17d90b5b7", environment="us-west4-gcp-free")
index = pinecone.Index(index_name = "law")
embed_model = "text-embedding-ada-002"

c:\Users\admin\AppData\Local\Programs\Python\Python311\Lib\site-packages\pinecone\index.py:4: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from tqdm.autonotebook import tqdm


In [ ]:
x = np.squeeze(df.Content.values)
sentence_list = [x for x in x]
sentence_list

In [2]:
sentence_list = ['hello']
Vectorstore, Only_Vectorstore = [], [] # consist of 20 vectors of chunk (sentence1 , sentence2, sentence3, ... ,sentence20)
for CHUNCK in sentence_list:
  embedding_ = openai.Embedding.create(input=CHUNCK,engine=embed_model)
  Vectorstore.append(embedding_)

for item in Vectorstore:
  for data_item in item['data']:
    Vector = data_item['embedding']
    Only_Vectorstore.append(Vector)

In [9]:
import torch
from scipy.spatial.distance import cosine
from transformers import AutoModel, AutoTokenizer

tokenizer = AutoTokenizer.from_pretrained("princeton-nlp/sup-simcse-roberta-large")
model = AutoModel.from_pretrained("princeton-nlp/sup-simcse-roberta-large")
torch.backends.cudnn.enabled = True
torch.cuda.is_available = lambda: True
torch.cuda.is_available()

batch_size = 4  # Set your desired batch size
num_sentences = len(sentence_list)
num_batches = (num_sentences + batch_size - 1) // batch_size
list = []
for batch_idx in range(num_batches):
    start_idx = batch_idx * batch_size
    end_idx = min((batch_idx + 1) * batch_size, num_sentences)
    batch_sentences = sentence_list[start_idx:end_idx]

inputs = tokenizer(
    sentence_list[0], padding=True, truncation=True, return_tensors="pt", add_special_tokens=True
)
with torch.no_grad():
    embeddings = model(**inputs, output_hidden_states=True, return_dict=True).last_hidden_state
    print(embeddings.shape)
list.append(embeddings.tolist())
print(embeddings)

    # Calculate cosine similarities in batch
    # cosine_sims = []
    # for i in range(len(batch_sentences)):
    #     for j in range(i + 1, len(batch_sentences)):
    #         cosine_sim = 1 - cosine(embeddings[i], embeddings[j])
    #         cosine_sims.append((batch_sentences[i], batch_sentences[j], cosine_sim))

    # # Print cosine similarities for this batch
    # for sentence1, sentence2, cosine_sim in cosine_sims:
    #     print(
    #         "Cosine similarity between \"%s\" and \"%s\" is: %.3f
    #         % (sentence1, sentence2, cosine_sim)
    #    )
print("Embeddings list:", list)


torch.Size([1, 3, 1024])
tensor([[[-0.6487,  0.7363, -2.3916,  ...,  0.1285, -1.4710,  1.6615],
         [-0.3432,  0.9137, -2.4742,  ..., -0.4358, -1.6807,  1.1677],
         [-0.6735,  0.8013, -2.1444,  ...,  0.2186, -1.6624,  1.3531]]])
Embeddings list: [[[[-0.6487467885017395, 0.7363238334655762, -2.3916361331939697, 0.5966275930404663, 0.11253970861434937, -0.8549439907073975, -0.48430389165878296, 2.320448160171509, 0.13184884190559387, 0.43731367588043213, 0.06848789751529694, 0.2701449394226074, -1.6754827499389648, -0.6434528231620789, 1.0372241735458374, 1.4858921766281128, 0.06622621417045593, 0.09643509984016418, 0.3453819751739502, -0.5251146554946899, -1.1769239902496338, 1.700296401977539, 0.4067019522190094, 0.28912025690078735, 0.018683120608329773, 1.0293184518814087, 1.118325114250183, -0.02458306774497032, -0.9539636373519897, 0.06917351484298706, -0.7159150838851929, 0.0665695071220398, -1.0216871500015259, -0.8770073652267456, 0.41242632269859314, -0.4782297313213

In [30]:
Embedding = pd.DataFrame(pd.Series(Only_Vectorstore), columns = ['Embedding'])
df['Embedding'] = Embedding
# df.insert(0, 'index', [str(k) for k in range(len(df))])

In [40]:
batch_size = 8
for i in range(0,len(df['Content']), batch_size):
    # set end position of batch
    i_end = min(i+batch_size, len(df['Content']))
    # get batch of lines and IDs
    lines_batch = df['Content'][i: i+batch_size]
    ids_batch = [str(n) for n in range(i, i_end)]
    # prep metadata and upsert batch
    meta = [{'Content': line} for line in lines_batch]
    to_upsert = list(zip(ids_batch, df.Embedding, meta))
    # upsert to Pinecone
    index.upsert(vectors=list(to_upsert), namespace= 'law_eng')

In [38]:
index.delete(delete_all=True, namespace="law_eng")

{}

In [41]:
index.describe_index_stats()

{'dimension': 1536,
 'index_fullness': 0.0,
 'namespaces': {'law': {'vector_count': 444}, 'law_eng': {'vector_count': 75}},
 'total_vector_count': 519}

In [47]:
df['Content'][0]

'Chapter 1: Terminology Section 1. Terminology and Definitions In this Code: “Dishonestly” means in such a manner as to enrich oneself or another by an unlawful means; “Public way” means a land course or watercourse for public transport, and includes a railway or tramway for public transit; “Public place” means any place to which the public is entitled to gain access; “Dwelling” means a place of habitation, such as an inhabited house, building, ship or floating house, and includes the vicinity of such place, whether encircled; “Weapon” includes an object which is not a weapon by nature but is used or intended to be used in causing any grievous bodily harm as if weapon; “Forcible attack” means an assault against the person or mind of a person, whether by physical force or by any other means, and includes any action by which a person places another in the state of being incapable of resistance, whether by intoxicant, hypnotization or any other similar means; “Document” means a paper or o

In [82]:
query_test = "rape"
test_emb = openai.Embedding.create(input=query_test,engine=embed_model)
index.query(
  vector=test_emb['data'][0]['embedding'], namespace = "law_eng",
  top_k= 5, include_metadata = True)

{'matches': [{'id': '4',
              'metadata': {'Content': 'Chapter 3: Punishments and Measures of '
                                      'Safety Part 1: Punishments Section 31. '
                                      'Multiple Offenders – Punishment of a '
                                      'Fine In case the Court shall pass '
                                      'judgment inflicting the punishment of '
                                      'fine on several offenders for the same '
                                      'offence and in the same case, the Court '
                                      'shall inflict the punishment of fine on '
                                      'every individual offender. Section 32. '
                                      'Use of Property in a Crime Any property '
                                      'is prescribed by the law that any '
                                      'person makes or processes to be an '
                             

In [54]:
limit = 3750
def retrieve(user_input):
    res = openai.Embedding.create(
        input=[user_input],
        engine=embed_model
    )
    # retrieve from Pinecone
    xq = res['data'][0]['embedding']
    # get relevant contexts
    res = index.query(xq, top_k = 5, namespace = "law_eng", include_metadata=True)
    contexts = [
        x['metadata']['Content'] for x in res['matches']
    ]
    # build our prompt with the retrieved contexts included
    prompt_start = (
        "LawBot, a GPT-based AI assistant designed specifically for answering legal questions and explaining complex legal terms.\
        Analysis the Question sector and provide a comprehensive and briefly answer in Answer section.The answer must address all aspects of the Question and support your argument with\
        appropriate legal references in Context section. Only answered by thai language only \n\n"+"Context:\n"
    )
    prompt_end = (
        f"\n\nQuestion: {user_input}\nAnswer:"
    )
    # append contexts until hitting limit
    for i in range(1, len(contexts)):
        if len("\n\n---\n\n".join(contexts[:i])) >= limit:
            prompt = (
                prompt_start +
                "\n\n---\n\n".join(contexts[:i-1]) +
                prompt_end
            )
            break
        elif i == len(contexts)-1:
            prompt = (
                prompt_start +
                "\n\n---\n\n".join(contexts) +
                prompt_end
            )
    return prompt

In [ ]:
มอเตอร์ไซด์สองคันชนกัน คันหนึ่งไม่ใส่หมวกกันน็อก อีกคันหนึ่งใส่หมวกกันน็อกแต่ขับย้อนศร ใครเป็นคนผิด

In [58]:
import requests, uuid, json

key = "041e6b79b96146eabafaf524d78911be"
endpoint = "https://api.cognitive.microsofttranslator.com"

location = "southeastasia"
path = '/translate'
constructed_url = endpoint + path

params = {
    'api-version': '3.0',
    'from': 'th',
    'to': ['en']
}

headers = {
    'Ocp-Apim-Subscription-Key': key,
    'Ocp-Apim-Subscription-Region': location,
    'Content-type': 'application/json',
    'X-ClientTraceId': str(uuid.uuid4())
}

def translator(input):
    body = [{
        'text': input
    }]
    request = requests.post(constructed_url, params=params, headers=headers, json=body)
    response = request.json()
    return response[0]["translations"][0]['text']


'Is it guilty to burn down your own house?'

In [76]:
translator("เผาบ้านเรือนมีความผิดตามข้อไหน")

"What's the offense of burning houses?"

In [74]:
setting = [{"role":"system","content":""},{"role":"user","content":""}]
def gpt3_completion(user_input, query_with_contexts, engine='gpt-35-turbo-16k', temp=0.5, top_p=1.0, tokens=2000, freq_pen=0.0, pres_pen=0.2):
    setting.append({'role':'system',"content":query_with_contexts})
    setting.append({'role':'user',"content":f"{gpt3_sum(user_input)}"})
    response = openai.ChatCompletion.create(engine=engine, temperature=temp, max_tokens=tokens,
      top_p=top_p, frequency_penalty=freq_pen, presence_penalty=pres_pen, messages = setting)
    print("LawBot:", response['choices'][0]['message']['content'])
    
def gpt3_sum(user_input, engine='gpt-35-turbo', temp=0.5, top_p=0.95, tokens=2000, freq_pen=0.0, pres_pen=0.2):
    setting.append({'role':'system',"content":"Summary the user input as a bullet , use only thai language"})
    setting.append({'role':'user',"content":f"{user_input}"})
    response = openai.ChatCompletion.create(engine=engine, temperature=temp, max_tokens=tokens,
      top_p=top_p, frequency_penalty=freq_pen, presence_penalty=pres_pen, messages = setting)
    # print("SumBot:", response['choices'][0]['message']['content'])
    return response['choices'][0]['message']['content']

while True:
  # เผาบ้านเรือนมีความผิดตามข้อไหน
  user_input = input()
  print("User:", user_input)
  if user_input == "quit":
    break
  query = retrieve(translator(user_input))
  # gpt3_sum(translator(user_input))
  response = gpt3_completion(translator(user_input), query)

User: เผาบ้านเรือนมีความผิดตามข้อไหน
LawBot: การเผาบ้านถือเป็นอาชญากรรมที่มีการกระทำความผิดตามกฎหมายไทย โดยอาศัยบทบัญญัติในส่วนที่เกี่ยวข้องกับความปลอดภัยภายในราชอาณาจักร ซึ่งอาศัยเนื้อหาของมาตรา 113 ในบทที่ 2 ของกฎหมายอาญาไทย ให้การกระทำการเผาบ้านเป็นการกระทำซึ่งเกี่ยวข้องกับการล้มล้างรัฐบาล โดยให้โทษประหารชีวิตหรือจำคุกตลอดชีวิตเป็นอันขาดหายนะ

Context:
Section 113. Overthrow Government Whoever, commits an act of violence or threatens to commit an act of violence in order to: Overthrow or change the Constitution; Overthrow the legislative power, the executive power or the judicial power of the Constitution, or nullify such power; or Separate the Kingdom or seize the power of administration in any part of the Kingdom, is said to commit insurrection, and shall be punished with death or imprisonment for life.

Analysis:
การเผาบ้านถือเป็นการกระทำความผิดที่เกี่ยวข้องกับความปลอดภัยภายในราชอาณาจักร และถูกกฎหมายอาญาไทยจัดว่าเป็นการล้มล้างรัฐบาล ตามมาตรา 113 ในบทที่ 2 ของกฎหมายอาญาไทย ในกรณีท

In [2]:
import os
os.environ["OPENAI_API_TYPE"] = "azure"
os.environ["OPENAI_API_VERSION"] = "2023-03-15-preview"
os.environ["OPENAI_API_BASE"] = "https://fusion-chatbot-demo.openai.azure.com/"
os.environ["OPENAI_API_KEY"] = "f07d23e10600494f8235496bbf5054b9"

In [3]:
from langchain.chains import RetrievalQA
from langchain.chat_models import AzureChatOpenAI
from langchain.callbacks import get_openai_callback
from langchain.schema import HumanMessage

query = "ฆ่าคนตายมีความผิดไหม"
# completion llm
BASE_URL = "https://fusion-chatbot-demo.openai.azure.com/"
API_KEY = "f07d23e10600494f8235496bbf5054b9"
DEPLOYMENT_NAME = "gpt-35-turbo-16k"
model = AzureChatOpenAI(
    openai_api_base=BASE_URL,
    openai_api_version="2023-05-15",
    deployment_name=DEPLOYMENT_NAME,
    openai_api_key=API_KEY,
    openai_api_type="azure",
)
with get_openai_callback() as cb:
    print(model(
        [
            HumanMessage(
                content="นายชื่ออะไร"
            )
        ]
    )
          )


content='ขออภัย ฉันไม่มีชื่อเนื่องจากฉันเป็นแค่ระบบคอมพิวเตอร์' additional_kwargs={} example=False


In [8]:
from langchain.prompts import PromptTemplate
template = """The following is a friendly conversation between a human and an AI.
Current conversation:
{history}
Human: {input}
AI Assistant:"""
prompt = PromptTemplate(input_variables=["history", "input"], template=template)

In [37]:
from langchain.chains import ConversationChain
from langchain.memory import ConversationSummaryBufferMemory

def memory(input):
    memory=ConversationSummaryBufferMemory(llm=model)
    memory.save_context({"input": "hi"}, {"output": "whats up"})

def talk(user_input):
    conversation_with_summary = ConversationChain(
        llm=model,
        memory=memory,
        prompt=prompt,
        verbose=True,
    )
    print(conversation_with_summary.predict(input=user_input))

In [10]:
from langchain.chains import ConversationChain
from langchain.memory import ConversationSummaryBufferMemory

class Chatbot():
    def __init__(self):
        self.memory = ConversationSummaryBufferMemory(llm=model)
    def talk(self, user_input):
        conversation_with_summary = ConversationChain(
            llm=model,
            memory= self.memory,
            prompt=prompt,
            verbose=True,
        )
        # self.memory.save_context({"input": user_input}, 
        #                          {"output": conversation_with_summary.predict(input=user_input)})
        print(conversation_with_summary.predict(input=user_input))


In [11]:
x = Chatbot()


In [13]:
x.talk("What did i ask you?")



> Entering new ConversationChain chain...
Prompt after formatting:
The following is a friendly conversation between a human and an AI.
Current conversation:
Human: 1 + 1 = ?
AI: 1 + 1 = 2.
Human: What did i ask you?
AI Assistant:

> Finished chain.
You asked me to solve the equation 1 + 1.
